# Simple Moving Average, with different windows shows the effects of smoothing
# Exponential Moving Average, more emphasis on recent price changes

In [31]:
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

ticker_symbol = "AMZN"
start = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
today = datetime.now().strftime("%Y-%m-%d")

stock_data = yf.download(ticker_symbol, start=start, end=today)

[*********************100%%**********************]  1 of 1 completed


In [47]:
stock_data.head(5)

,Open,High,Low,Close,Adj Close,Volume,SMA-3,SMA-10,EWM
Date,,,,,,,,,
2022-11-21,93.970001,95.019997,90.589996,92.459999,92.459999,84330300,NaN,92.459999,92.459999
2022-11-22,92.620003,93.349998,90.870003,93.199997,93.199997,62192000,NaN,92.829998,92.533999
2022-11-23,93.239998,94.580002,92.830002,94.129997,94.129997,59414700,93.263331,93.263331,92.693599
2022-11-25,93.790001,94.430000,93.070000,93.410004,93.410004,35088600,93.579999,93.299999,92.765239
2022-11-28,93.930000,96.400002,93.430000,93.949997,93.949997,74943100,93.829999,93.429999,92.883715


In [89]:
# Exponential moving average
# [ EMA_t = \alpha \times X_t + (1 - \alpha) \times EMA_{t-1} \]

alpha = 0.5
previous_ema = 0

sma_data = pd.DataFrame(index=stock_data.index)
def calculate_simple_moving_average(stock_data, window):
  slider = 0
  while window + slider <= stock_data.shape[0]:
    stock_data_for_window = stock_data.iloc[slider:slider + window]
    if len(stock_data_for_window) == window:  # Check if there are enough data points
        sma_value = stock_data_for_window["Adj Close"].mean()
        sma_data.loc[stock_data_for_window.index[-1], "SMA"] = sma_value
    else:
        sma_data.loc[stock_data_for_window.index[-1], "SMA"] = float('nan')
    slider += 1

calculate_simple_moving_average(stock_data, 3)


ema_data = pd.DataFrame(index=stock_data.index)
def calculate_exponential_moving_average(stock_data):
  for i in range(stock_data.shape[0]):
      current_price = stock_data.iloc[i]["Adj Close"]

      if i == 0:
        previous_ema = current_price

      current_ema = alpha * current_price + ( 1 - alpha ) * previous_ema
      ema_data.loc[stock_data.index[i], "EMA"] = current_ema
      previous_ema = current_ema

calculate_exponential_moving_average(stock_data)

In [90]:
# Visualise
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

# alpha
alpha = 0.1

stock_data["SMA-3"] = stock_data["Adj Close"].rolling(3).mean()
stock_data["SMA-10"] = stock_data["Adj Close"].rolling(10, min_periods=1).mean()
stock_data["EWM"] = stock_data["Adj Close"].ewm(alpha=alpha, adjust=False).mean()

fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["Adj Close"], name="Price"), secondary_y=False)
fig.add_trace(go.Scatter(x=sma_data.index, y=sma_data["SMA"], name="SMA"), secondary_y=True)
fig.add_trace(go.Scatter(x=ema_data.index, y=ema_data["EMA"], name="EMA"), secondary_y=True)
# fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["SMA-3"], name="SMA 3"), secondary_y=True)
# fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["SMA-10"], name="SMA 10"), secondary_y=True)
# fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data["EWM"], name="EWM"), secondary_y=True)

fig.update_layout(title_text="Trend Analysis")
fig.update_xaxes(title_text="Date")

fig.show()